# Assignment 4 : Body Landmark Localization using Hourglass Network


In [ ]:
# Seed
import torch
import numpy as np
import random

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### **4.1 Hourglass Module Implementation**

아래 **Fig. 3.**에 표현되어 있는 **Hourglass module**을 구현하고자 합니다. 이미 선언되어 있는 layer들을 이용하여 figure 상의 layer 구성과 동일하게 tensor가 forward 될 수 있도록 ```def forward``` 부분을 완성해주세요.


- Fig. 4. Right에 표현된 supervision layer는 해당 과제에서는 고려하지 않습니다.
- The figures are from [the original hourglass paper](https://arxiv.org/abs/1603.06937) [Newell et al.].

<img src='https://drive.google.com/uc?id=19-S7TwZ62joUR8W9031xjn3jMZyTevpw'  width="700">

<img src='https://drive.google.com/uc?id=1ols0VZ7TGZCMDM7sKzCJq3bByHsOU9up'  width="700">

아래의 코드는 Hourglass 모듈을 나타내는 클래스입니다. 위의 Figure를 참고하여 **TO DO** 과제를 채워주세요 :)

- **TO DO** : ```class Hourglass```는 하나의 Hourglass 모듈을 의미하며 이전에 선언한 ```class ResidualBlock```을 기본 convolution block으로 사용합니다. Hourglass 내부에 사용되는 layer는 이미 ```def __init__```에 선언이 되어 있지만 `**``def forward``` 부분은 완성되지 않아 선언된 layer들을 구성에 맞게 연결**해주어야 합니다. Fig. 3.을 참고하여 Hourglass 모듈을 올바르게 구현해주세요 :)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
  def __init__(self, num_channels=256):
    super(ResidualBlock, self).__init__()

    self.bn1 = nn.BatchNorm2d(num_channels)
    self.conv1 = nn.Conv2d(num_channels, num_channels//2, kernel_size=1, bias=True)

    self.bn2 = nn.BatchNorm2d(num_channels//2)
    self.conv2 = nn.Conv2d(num_channels//2, num_channels//2, kernel_size=3, stride=1,
                              padding=1, bias=True)

    self.bn3 = nn.BatchNorm2d(num_channels//2)
    self.conv3 = nn.Conv2d(num_channels//2, num_channels, kernel_size=1, bias=True)

    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    residual = x

    out = self.bn1(x)
    out = self.relu(out)
    out = self.conv1(out)

    out = self.bn2(out)
    out = self.relu(out)
    out = self.conv2(out)

    out = self.bn3(out)
    out = self.relu(out)
    out = self.conv3(out)

    out += residual

    return out

In [ ]:
class Hourglass(nn.Module):
  def __init__(self, block, num_channels=256):
    super(Hourglass, self).__init__()

    self.downconv_1 = block(num_channels)
    self.pool_1 = nn.MaxPool2d(kernel_size=2)
    self.downconv_2 = block(num_channels)
    self.pool_2 = nn.MaxPool2d(kernel_size=2)
    self.downconv_3 = block(num_channels)
    self.pool_3 = nn.MaxPool2d(kernel_size=2)
    self.downconv_4 = block(num_channels)
    self.pool_4 = nn.MaxPool2d(kernel_size=2)

    self.midconv_1 = block(num_channels)
    self.midconv_2 = block(num_channels)
    self.midconv_3 = block(num_channels)
    
    self.skipconv_1 = block(num_channels)
    self.skipconv_2 = block(num_channels)
    self.skipconv_3 = block(num_channels)
    self.skipconv_4 = block(num_channels)

    self.upconv_1 = block(num_channels)
    self.upconv_2 = block(num_channels)
    self.upconv_3 = block(num_channels)
    self.upconv_4 = block(num_channels)

  def forward(self, x):
    x1 = self.downconv_1(x)
    x  = self.pool_1(x1)

    '''======================================================='''
    '''======================== TO DO ========================'''
    x2 = self.downconv_2(x)
    x  = self.pool_2(x2)
    x3 = self.downconv_3(x)
    x  = self.pool_3(x3)
    x4 = self.downconv_4(x)
    x  = self.pool_4(x4)

    x = self.midconv_1(x)
    x = self.midconv_2(x)
    x = self.midconv_3(x)

    x4 = self.skipconv_1(x4)
    x = F.upsample(x, scale_factor=2)
    x = x + x4
    x = self.upconv_1(x)

    x3 = self.skipconv_1(x3)
    x = F.upsample(x, scale_factor=2)
    x = x + x3
    x = self.upconv_2(x)

    x2 = self.skipconv_1(x2)
    x = F.upsample(x, scale_factor=2)
    x = x + x2
    x = self.upconv_3(x)

    x1 = self.skipconv_1(x1)
    x = F.upsample(x, scale_factor=2)
    x = x + x1
    x = self.upconv_4(x)
    '''======================== TO DO ========================'''
    '''======================================================='''

    return x

----
[torchsummary](https://github.com/sksq96/pytorch-summary)는 PyTorch로 구현한 네트워크를 직관적으로 확인할 수 있는 라이브러리입니다.

해당 라이브러리를 이용하여 각 feature map의 dimension과 각각의 layer가 몇개의 parameter 수를 가지고 있는지 확인해 봅시다!

In [ ]:
# Let's summary the implemented hourglass architecture using torchsummary library.
hg = Hourglass(ResidualBlock)

from torchsummary import summary
summary(hg, input_size=(256,64,64), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm2d-1          [-1, 256, 64, 64]             512
              ReLU-2          [-1, 256, 64, 64]               0
            Conv2d-3          [-1, 128, 64, 64]          32,896
       BatchNorm2d-4          [-1, 128, 64, 64]             256
              ReLU-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 128, 64, 64]         147,584
       BatchNorm2d-7          [-1, 128, 64, 64]             256
              ReLU-8          [-1, 128, 64, 64]               0
            Conv2d-9          [-1, 256, 64, 64]          33,024
    ResidualBlock-10          [-1, 256, 64, 64]               0
        MaxPool2d-11          [-1, 256, 32, 32]               0
      BatchNorm2d-12          [-1, 256, 32, 32]             512
             ReLU-13          [-1, 256, 32, 32]               0
           Conv2d-14          [-1, 128,

----
아래는 네트워크 구현이 정확하게 되었는지를 확인하기 위한 코드입니다.

<br>Assignment1와 마찬가지로 아래의 코드는 **Hourglass 모듈의 중간 feature map들의 shape**을 바탕으로 일련의 연산을 수행하여 **하나의 값**을 계산합니다.
<br>채점을 위하여 아래의 코드 결과로 얻은 값을 **edwith에 제출**해주세요 :)

(주의 : 정확한 채점을 위하여 아래 코드는 수정하지 마세요!)

<br>예를 들어, 아래와 같은 실행 결과를 얻으셨다면 edwith 퀴즈에 7777을 선택해주세요.
```python
"Your answer is : 7777"
```

In [ ]:
import base64, copy

class Calculator:
  '''
  NOTE : DO NOT MODIFY THE CODE BELOW.
  '''
  def __init__(self, model):
    self.answer = 0
    modules = [b'c2tpcGNvbnZfMg==\n', b'dXBjb252XzM=\n']
    layer = b'Y29udjI=\n'
    for m in modules:
      self.hook = model._modules[base64.decodebytes(m).decode()]._modules[base64.decodebytes(layer).decode()].register_forward_hook(self.hook_fn)
    
  def hook_fn(self, module, input, output):
    self.answer += self._get_answer(output)
  
  def _get_answer(self, l):
    _, A, B, C = l.shape
    return A*(B-C//3)
    
  def unregister_forward_hook(self):
    self.hook.remove()
  

def calc_anwser(model):
  # NOTE : DO NOT MODIFY THE CODE BELOW.
  model_test = copy.deepcopy(model)
  ans_calculator = Calculator(model_test)

  x = torch.rand(1,256,64,64)
  model_test(x)

  print("Your answer is : %d" % ans_calculator.answer)

In [ ]:
calc_anwser(hg)

Your answer is : 2816


----
### **4.2 Human Pose Estimation**

[Stacked Hourglass Network](https://arxiv.org/abs/1603.06937)를 이용하여 human pose estimation task를 수행하여 봅시다!

<img src='https://drive.google.com/uc?id=1gJPaBX8uVWY9FnNRf2H3rmP1xYsd73eR'  width="900">

##### **>>> 4.2.1 Stacked Hourglass Network**
아래 코드는 stacked hourglass network의 전체 코드입니다. ([원본 github 링크](https://github.com/bearpaw/pytorch-pose))

- 3.1에서 Hourglass 모듈을 구현할 때 일일이 layer를 쌓는 것 대신에 for loop와 [nn.ModuleList](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html)를 이용하여 더욱 직관적이고 명료한 코드 작성이 가능하다는 것도 한번 확인해보세요 :)

In [ ]:
'''
Hourglass network inserted in the pre-activated Resnet
Use lr=0.01 for current version
(c) YANG, Wei
'''
import torch.nn as nn
import torch.nn.functional as F

# from .preresnet import BasicBlock, Bottleneck


__all__ = ['HourglassNet', 'hg']

class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()

        self.bn1 = nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=True)
        self.bn3 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 2, kernel_size=1, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.bn1(x)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        return out


class Hourglass(nn.Module):
    def __init__(self, block, num_blocks, planes, depth):
        super(Hourglass, self).__init__()
        self.depth = depth
        self.block = block
        self.hg = self._make_hour_glass(block, num_blocks, planes, depth)

    def _make_residual(self, block, num_blocks, planes):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(planes*block.expansion, planes))
        return nn.Sequential(*layers)

    def _make_hour_glass(self, block, num_blocks, planes, depth):
        hg = []
        for i in range(depth):
            res = []
            for j in range(3):
                res.append(self._make_residual(block, num_blocks, planes))
            if i == 0:
                res.append(self._make_residual(block, num_blocks, planes))
            hg.append(nn.ModuleList(res))
        return nn.ModuleList(hg)

    def _hour_glass_forward(self, n, x):
        up1 = self.hg[n-1][0](x)
        low1 = F.max_pool2d(x, 2, stride=2)
        low1 = self.hg[n-1][1](low1)

        if n > 1:
            low2 = self._hour_glass_forward(n-1, low1)
        else:
            low2 = self.hg[n-1][3](low1)
        low3 = self.hg[n-1][2](low2)
        up2 = F.interpolate(low3, scale_factor=2)
        out = up1 + up2
        return out

    def forward(self, x):
        return self._hour_glass_forward(self.depth, x)


class HourglassNet(nn.Module):
    '''Hourglass model from Newell et al ECCV 2016'''
    def __init__(self, block, num_stacks=2, num_blocks=4, num_classes=16):
        super(HourglassNet, self).__init__()

        self.inplanes = 64
        self.num_feats = 128
        self.num_stacks = num_stacks
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=True)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_residual(block, self.inplanes, 1)
        self.layer2 = self._make_residual(block, self.inplanes, 1)
        self.layer3 = self._make_residual(block, self.num_feats, 1)
        self.maxpool = nn.MaxPool2d(2, stride=2)

        # build hourglass modules
        ch = self.num_feats*block.expansion
        hg, res, fc, score, fc_, score_ = [], [], [], [], [], []
        for i in range(num_stacks):
            hg.append(Hourglass(block, num_blocks, self.num_feats, 4))
            res.append(self._make_residual(block, self.num_feats, num_blocks))
            fc.append(self._make_fc(ch, ch))
            score.append(nn.Conv2d(ch, num_classes, kernel_size=1, bias=True))
            if i < num_stacks-1:
                fc_.append(nn.Conv2d(ch, ch, kernel_size=1, bias=True))
                score_.append(nn.Conv2d(num_classes, ch, kernel_size=1, bias=True))
        self.hg = nn.ModuleList(hg)
        self.res = nn.ModuleList(res)
        self.fc = nn.ModuleList(fc)
        self.score = nn.ModuleList(score)
        self.fc_ = nn.ModuleList(fc_)
        self.score_ = nn.ModuleList(score_)

    def _make_residual(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=True),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _make_fc(self, inplanes, outplanes):
        bn = nn.BatchNorm2d(inplanes)
        conv = nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=True)
        return nn.Sequential(
                conv,
                bn,
                self.relu,
            )

    def forward(self, x):
        out = []
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.maxpool(x)
        x = self.layer2(x)
        x = self.layer3(x)

        for i in range(self.num_stacks):
            y = self.hg[i](x)
            y = self.res[i](y)
            y = self.fc[i](y)
            score = self.score[i](y)
            out.append(score)
            if i < self.num_stacks-1:
                fc_ = self.fc_[i](y)
                score_ = self.score_[i](score)
                x = x + fc_ + score_

        return out

In [ ]:
model = HourglassNet(Bottleneck, num_stacks=1, num_blocks=2, num_classes=22).cuda()

----
##### **>>> 4.2.2 Custom Body Landmark Dataset**
해당 과제에서는 이미지 속 인물의 여러 신체 부위를 keypoint 형태로 예측하는 네트워크를 학습시키고자 합니다. (학습 시간 단축을 위해 일부 데이터만 사용)

과제를 수행하기 앞서 별도로 전달해드린 **데이터셋 활용 가이드**를 따라 **```APY191016001_Body_Landmarks_Dataset_Shared_Subset_20p.zip```** 압축 파일을 구글 드라이브에 **바로가기 추가**해주세요 :)


<br></br>**주의!** 해당 과정은 **데이터 저작권 보호**를 위해 로컬로 직접 데이터를 <U>**다운로드 받는 것을 금지**</U>하기 때문입니다. 또한 교육이 종료된 이후에 해당 데이터셋을 **구글 드라이브에서 파기**할 것을 원칙으로 합니다.
<img src='https://drive.google.com/uc?id=14_MWEN5u9mAvO1le_wYVPWdeduUnwW1B'  width="700">

In [ ]:
# Mount the google drive to access the dataset.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# 저장하신 압축 파일의 경로에 맞게 아래의 압축 해제 명령어를 수정해주세요. (!tar -zxvf 압축파일_경로 -C 저장할_폴더)

!unzip /content/gdrive/MyDrive/APY191016001_Body_Landmarks_Dataset_Shared_Subset_20p.zip -d /content/BodyLandmarkData

Archive:  /content/gdrive/MyDrive/APY191016001_Body_Landmarks_Dataset_Shared_Subset_20p.zip


In [ ]:
# Hyper-paramter Settings
data_root = '/content/BodyLandmarkData/data'
log_dir   = '/content/BodyLandmarkData/log'

epochs = 3
batch_size = 8
lr = 1e-3
input_size = 320

학습을 위해서는 제공받은 데이터셋의 landmark 정보를 parsing하여 heatmap 형태로 나타내어야 합니다.

Gaussin heatmap 형태로 keypoint를 나타내기 위하여 7강 강의 자료의 24번째 슬라이드를 참고하여 **TO DO**를 채워주세요 

In [ ]:
# Dataset
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import os
import cv2
import json
import numpy as np
from glob import glob

class BodyLandmarkDataset(Dataset):
  def __init__(self, data_root, is_Train=True, input_size=224, transform=None):
    super(BodyLandmarkDataset, self).__init__()

    self.img_list = self._load_img_list(data_root, is_Train)

    self.len = len(self.img_list)
    self.input_size = input_size
    self.hm_size = input_size//4
    self.transform = transform
    
    self.n_landmarks = 22
    self.sigma = 1.5

  def __getitem__(self, index):
    img_path = self.img_list[index]
    anno_path = img_path.replace('.jpg', '.json')
    
    # Image Loading
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/255.
    
    org_size = img.shape[:2]

    if self.transform:
      img = self.transform(img)

    # Ground Truth
    heatmap = self._get_heatmaps_from_json(anno_path, org_size)

    return img, heatmap

  def __len__(self):
    return self.len
  
  def _load_img_list(self, data_root, is_Train):
    # Change the name of directory which has inconsistent naming rule.
    full_img_list = glob(os.path.join(data_root, 'single', '*', '*color.jpg'))
    
    # ID < 400 for Training
    # 400 < ID for Validation
    if is_Train:
      return [path for path in full_img_list if (self._load_img_ID(path) < 400)]
    else:
      return [path for path in full_img_list if (400 < self._load_img_ID(path))]

  def _load_img_ID(self, path):
    return int(path.split(os.sep)[-2].strip('id_1'))

  def _get_heatmaps_from_json(self, anno_path, org_size):
    # Parse point annotation
    with open(anno_path, 'r') as json_file:
      pts = json.load(json_file)
    pts = np.array([(pt['pt_x'], pt['pt_y']) for pt in pts['DataList'][0]['coordinates']])

    pts[:,0] = pts[:,0] / org_size[1] * self.hm_size
    pts[:,1] = pts[:,1] / org_size[0] * self.hm_size

    heatmap = np.zeros((self.n_landmarks, self.hm_size, self.hm_size), dtype=np.float32)
    for i, pt in enumerate(pts):
      heatmap[i] = self._draw_labelmap(heatmap[i], org_size, pt, self.sigma)
    
    return heatmap

  def _draw_labelmap(self, heatmap, org_size, pt, sigma):
    # Draw a 2D gaussian
    # Adopted from https://github.com/anewell/pose-hg-train/blob/master/src/pypose/draw.py
    H, W = heatmap.shape[:2]

    # Check that any part of the gaussian is in-bounds
    ul = [int(pt[0] - 3 * sigma), int(pt[1] - 3 * sigma)]
    br = [int(pt[0] + 3 * sigma + 1), int(pt[1] + 3 * sigma + 1)]
    if (ul[0] >= heatmap.shape[1] or ul[1] >= heatmap.shape[0] or
            br[0] < 0 or br[1] < 0):
        # If not, just return the image as is
        return heatmap, 0

    # Generate gaussian
    size = 6 * sigma + 1
    x = np.arange(0, size, 1, float)
    y = x[:, np.newaxis]
    x0 = y0 = size // 2
    # The gaussian is not normalized, we want the center value to equal 1

    '''======================================================='''
    '''======================== TO DO ========================'''
    g = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
    '''======================== TO DO ========================'''
    '''======================================================='''

    # Usable gaussian range
    g_x = max(0, -ul[0]), min(br[0], heatmap.shape[1]) - ul[0]
    g_y = max(0, -ul[1]), min(br[1], heatmap.shape[0]) - ul[1]
    # Image range
    heatmap_x = max(0, ul[0]), min(br[0], heatmap.shape[1])
    heatmap_y = max(0, ul[1]), min(br[1], heatmap.shape[0])

    heatmap[heatmap_y[0]:heatmap_y[1], heatmap_x[0]:heatmap_x[1]] = g[g_y[0]:g_y[1], g_x[0]:g_x[1]]
    return heatmap
    
    return anno_path

In [ ]:
# Dataset and Data Loader
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((input_size, input_size)),
    transforms.Normalize(mean=MEAN,
                          std=STD)
])

train_dataset = BodyLandmarkDataset(data_root, is_Train=True, input_size=input_size, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True, shuffle=True)

valid_dataset = BodyLandmarkDataset(data_root, is_Train=False, input_size=input_size, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)

In [ ]:
# Misc

class AverageMeter(object):
  """Computes and stores the average and current value"""
  def __init__(self):
      self.reset()

  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

----
#### **>>> 4.2.3 Training**

```BodyLandmarkDataset```을 활용하여 Hourglass network를 학습할 시간입니다.

- **TO DO Main (1)** : VGG-11을 본격적으로 학습하는 과정입니다. 주석에 적힌 내용을 따라 loss function인 ```criterion```과 ```optimizer```를 활용하여 빈 부분을 채워주세요.

- **TO DO Main (2)** : 학습된 VGG-11을 validation dataset에 대해 평가하는 과정입니다. Validation 과정에서는 <U>gradient 계산과 backpropagation이 필요 없다</U>는 것에 주목하여 빈 부분을 채워주세요.

In [ ]:
# Loss function and Optimizer
from torch.optim import Adam

criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=lr)

In [ ]:
# Main
os.makedirs(log_dir, exist_ok=True)

with open(os.path.join(log_dir, 'train_log.csv'), 'w') as log:
  for epoch in range(epochs):
    train_loss, valid_loss = AverageMeter(), AverageMeter()

    # Training
    for iter, (img, hm_gt) in enumerate(train_loader):
      '''================================================================'''
      '''======================== TO DO Main (1) ========================'''
      # optimizer에 저장된 미분값을 0으로 초기화
      optimizer.zero_grad()

      # GPU 연산을 위해 이미지와 정답 tensor를 GPU로 보내기 (필요한 경우, 변수의 type도 수정해주세요)
      img, hm_gt = img.float().cuda(), hm_gt.float().cuda()

      # 모델에 이미지 forward
      pred_logit = model(img)

      # loss 값 계산
      loss = 0
      for pred in pred_logit:
        loss += criterion(pred, hm_gt)

      # Backpropagation
      loss.backward()
      optimizer.step()
      '''======================== TO DO Main (1) ========================'''
      '''================================================================'''

      # Log Update
      train_loss.update(loss.item(), len(img))
      print("\rEpoch [%3d/%3d] | Iter [%3d/%3d] | Train Loss %.4f" % (epoch+1, epochs, iter+1, len(train_loader), train_loss.avg), end='')

    # Validation
    for iter, (img, hm_gt) in enumerate(valid_loader):
      '''================================================================'''
      '''======================== TO DO Main (2) ========================'''
      # GPU 연산을 위해 이미지와 정답 tensor를 GPU로 보내기 (필요한 경우, 변수의 type도 수정해주세요)
      img, hm_gt = img.float().cuda(), hm_gt.float().cuda()

      # 모델에 이미지 forward (gradient 계산 X)
      with torch.no_grad():
        pred_logit = model(img)

      # loss 값 계산
      loss = 0
      for pred in pred_logit:
        loss += criterion(pred, hm_gt)
      '''======================== TO DO Main (2) ========================'''
      '''================================================================'''

      # Log Update
      valid_loss.update(loss.item(), len(img))
 
    print("\nEpoch [%3d/%3d] | Valid Loss %.4f" % (epoch+1, epochs, valid_loss.avg))
    
    # Log Writing
    log.write('%d,%.4f,%.4f\n'%(epoch, train_loss.avg, valid_loss.avg))

Epoch [  1/  3] | Iter [305/321] | Train Loss 0.0026

KeyboardInterrupt: ignored

#### **>>> 4.3.3 Visualization**
학습된 모델을 바탕으로 샘플 이미지에 대한 keypoint 예측 결과를 시각화하는 단계입니다.

- **TO DO** : 아래의 시각화 코드를 활용하여 샘플 이미지에 대한 예측 결과를 시각화해주세요. ```matplotlib```을 이용하여 시각화한 그래프가 해당 colab notebook 파일에 남아있어야 하며 해당 과정을 마치신 뒤에 edwith의 댓글로 colab link를 남겨주세요. (아래 예시 그림 참고) <img src='https://drive.google.com/uc?id=1zCiRG-vQ2lSantORSmQYbqH75rB9Rb5f'  width="400">


- **TO DO Main** : 주석을 참고하여 inference를 위한 코드를 완성해주세요.

- **TO DO Decoding** : 예측된 heatmap에서 좌표값 (x,y)를 얻어내는 코드를 완성해주세요.
<br>(1) ```pred_hm``` 변수는 (channels, height, width) shape을 가집니다.
<br>(2) ```hm``` 변수는 ```pred_hm```의 각 channel을 나타내며 (height, width) shape을 가집니다.

In [ ]:
import matplotlib.pyplot as plt

n_vis = 5

# Visualize the result of validation dataset
for iter, (imgs, hm_gts) in enumerate(train_loader):
  '''============================================================'''
  '''======================== TO DO Main ========================'''
  # GPU 연산을 위해 이미지 tensor를 GPU로 보내기 (필요한 경우, 변수의 type도 수정해주세요)
  imgs = imgs.float().cuda()
  
  # 모델에 이미지 forward (gradient 계산 X)
  with torch.no_grad():
    preds = model(imgs)[-1].cpu().numpy()
  '''======================== TO DO Main ========================'''
  '''============================================================'''


  # for each sample in a batch
  imgs = imgs.cpu().numpy()
  for img, pred_hm in zip(imgs, preds):
    # Re-convert pre-processed input image to original format
    img = np.moveaxis(img, 0, -1)
    img = (img * STD) + MEAN
    img = (img*255).astype(np.uint8).copy()

    for hm in pred_hm:
      '''======================================================='''
      '''==================== TO DO Decoding ==================='''
      y, x = np.where(hm == hm.max())
      '''==================== TO DO Decoding ==================='''
      '''======================================================='''
      cv2.circle(img, (x[0]*4, y[0]*4), 3, (255,0,0), -1)
    
    plt.imshow(img)
    plt.show()
  

  if iter == (n_vis-1):
    break